# IMPORTAR LIBRERIAS

In [1]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

C:\Users\OT\AppData\Local\Temp\ipykernel_9532\1313406465.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# FUNCIONES

Se definen las funciones que se utilizarán para la depuración del dataset.

In [2]:
def filtrar_datos_en_partes(filepath, columna_alimentos, alimentos_seleccionados, chunk_size=10000):
    """
    Filtra datos en partes para evitar cargar el archivo completo en memoria.
    
    :param filepath: Ruta del archivo CSV
    :param columna_alimentos: Nombre de la columna con los alimentos
    :param alimentos_seleccionados: Lista de alimentos a seleccionar
    :param chunk_size: Número de filas por cada chunk
    :return: DataFrame filtrado
    """
    # Crear un DataFrame vacío para almacenar los resultados
    df_filtrado = pd.DataFrame()
    
    # Leer el archivo en partes (chunks)
    for chunk in pd.read_csv(filepath, chunksize=chunk_size):
        # Filtrar el chunk actual
        chunk_filtrado = chunk[chunk[columna_alimentos].isin(alimentos_seleccionados)]
        # Añadir el chunk filtrado al DataFrame final
        df_filtrado = pd.concat([df_filtrado, chunk_filtrado], ignore_index=True)
    
    return df_filtrado

In [3]:
def convertir_a_fecha(df):
    """
    Esta función toma las columnas 'Año' y 'Mes' de un DataFrame, las combina para crear una nueva
    columna 'Fecha' en formato 'YYYY-MM'

    Parámetros:
    - df (pd.DataFrame): El DataFrame que contiene las columnas 'Año' y 'Mes', las cuales se 
      utilizan para crear la columna 'Fecha'.

    Retorna:
    - pd.DataFrame: El DataFrame original con una nueva columna 'Fecha', devolviendo Año, Mes y Fecha en tipo str.
    """
    
    df['Año'] = df['Año'].astype(str).str.extract('(\d{4})')  
    df['Mes'] = df['Mes'].astype(str).str.zfill(2)  
    df['Fecha'] = df['Año'] + '-' + df['Mes']
    
    return df


# CARGAR DATOS

El conjunto de datos abarca una amplia variedad de categorías de alimentos. Para el análisis y modelización, nos enfocaremos exclusivamente en un grupo específico de alimentos, seleccionando aquellos que reflejan los patrones de consumo más relevantes a nivel nacional. Nuestro objetivo es identificar y trabajar con los grupos de alimentos más representativos en términos de consumo alimentario. Los grupos seleccionados son: frutas, hortalizas, pastas, arroz, pan, carne, pescado, huevos, legumbres, leche líquida, derivados lácteos y platos preparados.

Antes de extraer estos grupos del conjunto de datos original, es necesario observar cómo están referenciados. Los diferentes archivos originales utilizados para crear el dataset contenían los nombres de los grupos de alimentos que se desean analizar con denominaciones diversas. En primer lugar, se realiza un filtrado general de los datos y, posteriormente, se aplica una transformación de nombres para unificar las categorías.

In [4]:
# Definir el directorio de salida
directorioSalida = './Dataset/Dataset_final/'

# Definir los archivos
datosNacional_csv = directorioSalida + 'nacional_combinados.csv'
datosCcaa_csv = directorioSalida + 'datos_combinados_CCAA.csv'

#Definir el grupo de alimentos con el que se quiero trabajar. 
seleccion_alimentos = [
    'T.FRUTAS FRESCAS', 'T.HORTALIZAS FRESCAS', 'TOTAL PASTAS', 'ARROZ', 'PAN', 
    'TOTAL CARNE', 'TOTAL PESCA', 'T.HUEVOS KGS', 'LEGUMBRES', 'PLATOS PREPARADOS', 
    'TOTAL LECHE LIQUIDA', 'DERIVADOS LACTEOS','TOTAL FRUTAS FRESCAS','TOTAL HORTALIZAS FRES','HUEVOS KGS',
    'HUEVOS (Kgs)'
]

df_nacional_filtrado = filtrar_datos_en_partes(datosNacional_csv , 'Alimentos', seleccion_alimentos)
df_ccaa_filtrado = filtrar_datos_en_partes(datosCcaa_csv, 'Alimentos', seleccion_alimentos)

In [5]:
# Reemplazar valores específicos en la columna 'Alimentos'
df_nacional_filtrado['Alimentos'] = df_nacional_filtrado['Alimentos'].replace({
    'TOTAL FRUTAS FRESCAS': 'T.FRUTAS FRESCAS',
    'TOTAL HORTALIZAS FRES': 'T.HORTALIZAS FRESCAS',
    'HUEVOS KGS':'T.HUEVOS KGS',
    'HUEVOS (Kgs)':'T.HUEVOS KGS'
})


df_ccaa_filtrado['Alimentos'] = df_ccaa_filtrado['Alimentos'].replace({
    'TOTAL FRUTAS FRESCAS': 'T.FRUTAS FRESCAS',
    'TOTAL HORTALIZAS FRES': 'T.HORTALIZAS FRESCAS',
    'HUEVOS KGS':'T.HUEVOS KGS',
    'HUEVOS (Kgs)':'T.HUEVOS KGS'
})

# DEPURAR DATOS

### TIPOLOGIA VARIABLES

Podemos observar como todas las variables estan bien definidas. Pero Mes lo encontramos en todos los dataframes en formato object, en nuestro caso, deseamos traspasarlo a int. Podemos observar también valores nulos en este primner análisis.

También se observa en el dataset por ccaa como tenemos definido para el precio medio y volumen diferentes nombres duplicando las columnas, se unificará en una sola. 

In [6]:
df_nacional_filtrado.info(),df_ccaa_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3636 entries, 0 to 3635
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Alimentos                    3636 non-null   object 
 1   CONSUMO X CAPITA             3636 non-null   float64
 2   GASTO X CAPITA               3636 non-null   float64
 3   PENETRACION (%)              3636 non-null   float64
 4   PRECIO MEDIO kg ó litros     3636 non-null   float64
 5   VALOR (Miles Euros)          3636 non-null   float64
 6   VOLUMEN (Miles kg ó litros)  3636 non-null   float64
 7   Año                          3636 non-null   int64  
 8   Mes                          3636 non-null   object 
dtypes: float64(6), int64(1), object(2)
memory usage: 255.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61524 entries, 0 to 61523
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------              

(None, None)

In [7]:
# Unificar las columnas 'PRECIO MEDIO kg ó litros' y 'PRECIO MEDIO kg'
df_ccaa_filtrado['PRECIO MEDIO kg ó litros'] = df_ccaa_filtrado['PRECIO MEDIO kg ó litros'].combine_first(df_ccaa_filtrado['PRECIO MEDIO kg'])

# Unificar las columnas 'VOLUMEN (Miles kg ó litros)' y 'VOLUMEN (Miles Kg)'
df_ccaa_filtrado['VOLUMEN (Miles kg ó litros)'] = df_ccaa_filtrado['VOLUMEN (Miles kg ó litros)'].combine_first(df_ccaa_filtrado['VOLUMEN (Miles Kg)'])

# Eliminar las columnas originales no unificadas
df_ccaa_filtrado = df_ccaa_filtrado.drop(columns=['PRECIO MEDIO kg', 'VOLUMEN (Miles Kg)'])

In [8]:
df_ccaa_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61524 entries, 0 to 61523
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Alimentos                    61524 non-null  object 
 1   CONSUMO X CAPITA             61524 non-null  float64
 2   GASTO X CAPITA               61524 non-null  float64
 3   PENETRACION (%)              61524 non-null  float64
 4   PRECIO MEDIO kg ó litros     61524 non-null  float64
 5   VALOR (Miles Euros)          61524 non-null  float64
 6   VOLUMEN (Miles kg ó litros)  61524 non-null  float64
 7   Mes                          61524 non-null  object 
 8   Año                          61524 non-null  int64  
 9   CCAA                         61524 non-null  object 
dtypes: float64(6), int64(1), object(3)
memory usage: 4.7+ MB


Antes de transformar el mes a numérico, observamos como se presenta el mismo en cada dataset. Como se puede observar se presenta en distintos formatos, iniciando en mayuscula o míniscula y con algunos errores de escritura como en septeimbre.

In [9]:
df_nacional_filtrado['Mes'].unique(),df_ccaa_filtrado['Mes'].unique()

(array(['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio',
        'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre',
        'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio',
        'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre',
        'septeimbre'], dtype=object),
 array(['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio',
        'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre',
        'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio',
        'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre',
        'septeimbre'], dtype=object))

In [10]:
meses_map = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'septeimbre':9, #afecta específicamente a 2002
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12
}


# Convertir los nombres de los meses a minúsculas y luego a números usando el diccionario para nacional y ccaa
df_nacional_filtrado['Mes'] = df_nacional_filtrado['Mes'].str.lower().map(meses_map)
df_ccaa_filtrado['Mes'] = df_ccaa_filtrado['Mes'].str.lower().map(meses_map)


In [11]:
#Comprobar que esten bien definidos los meses
df_nacional_filtrado['Mes'].unique(),df_ccaa_filtrado['Mes'].unique()

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64))

In [12]:
df_lista = [df_nacional_filtrado, df_ccaa_filtrado]
df_lista = [convertir_a_fecha(df) for df in df_lista]

In [13]:
#Unificamos también nombres de comunidades autónomas
ccaa_mapping = {
    'ILLES BALEARS': 'BALEARES',
    'COMUNITAT VALENCIANA': 'VALENCIA',
    'REGIÓN DE MURCIA': 'MURCIA',
    'COMUNIDAD DE MADRID': 'MADRID',
    'CASTILLA - LA MANCHA': 'CASTILLA-LA MANCHA',
    'PRINCIPADO DE ASTURIAS': 'ASTURIAS',
    'CANTABRIA': 'CANTABRIA',
    'PAIS VASCO': 'PAIS VASCO',
    'LA RIOJA': 'LA RIOJA',
    'C. FORAL DE NAVARRA': 'NAVARRA',
    'CANARIAS': 'CANARIAS',
    'ARAGON': 'ARAGÓN',
    'T.ANDALUCIA': 'ANDALUCÍA',
    'CASTILLA LA MANCHA': 'CASTILLA-LA MANCHA',
    'CASTILLA LEON': 'CASTILLA Y LEÓN',
    'CASTILLA Y LEON': 'CASTILLA Y LEÓN',
    'RIOJA': 'LA RIOJA',
    'EXTEMADURA':'EXTREMADURA',
    'ANDALUCIA':'ANDALUCÍA',
}

df_ccaa_filtrado['CCAA'] = df_ccaa_filtrado['CCAA'].replace(ccaa_mapping)

### DATOS ATÍPICOS

A continuación se realiza el estudio de datos atípicos de los distintos datasets. Dado que los datasets incluyen múltiples series temporales correspondientes a los distintos grupos de alimentos seleccionados, no es adecuado aplicar un análisis de valores atípicos directamente sobre el conjunto completo del dataset. En lugar de eso, es necesario segmentar el dataset en series temporales individuales basadas en cada grupo de alimento específico. Esto permitirá realizar un análisis de valores atípicos más preciso y relevante, ya que los patrones y comportamientos de cada grupo de alimento pueden variar significativamente. 

Además, en el análisis de los datos a nivel de comunidad autónoma, es importante reconocer que las variaciones entre las distintas variables pueden reflejar patrones específicos de cada región y no necesariamente deben considerarse como datos atípicos. Para evitar tratar estas variaciones regionales como anomalías, se realizará un desglose detallado del tratamiento de valores atípicos no solo a nivel de alimento, sino también a nivel de comunidad autónoma. 

In [14]:
def atipicosAmissing(varaux):
    """
    Esta función identifica valores atípicos en una serie de datos y los reemplaza por NaN.
    
    Datos de entrada:
    - varaux: Serie de datos en la que se buscarán valores atípicos.
    
    Datos de salida:
    - Una nueva serie de datos con valores atípicos reemplazados por NaN.
    - El número de valores atípicos identificados.
    """
    
    # Verifica si la distribución de los datos es simétrica o asimétrica
    if abs(varaux.skew()) < 1:
        # Si es simétrica, calcula los valores atípicos basados en la desviación estándar
        criterio1 = abs((varaux - varaux.mean()) / varaux.std()) > 3
    else:
        # Si es asimétrica, calcula la Desviación Absoluta de la Mediana (MAD) y los valores atípicos
        mad = sm.robust.mad(varaux, axis=0)
        criterio1 = abs((varaux - varaux.median()) / mad) > 8
    
    # Calcula los cuartiles 1 (Q1) y 3 (Q3) para determinar el rango intercuartílico (H)
    qnt = varaux.quantile([0.25, 0.75]).dropna()
    Q1 = qnt.iloc[0]
    Q3 = qnt.iloc[1]
    H = 3 * (Q3 - Q1)
    
    # Identifica valores atípicos que están fuera del rango intercuartílico
    criterio2 = (varaux < (Q1 - H)) | (varaux > (Q3 + H))
    
    # Crea una copia de la serie original y reemplaza los valores atípicos por NaN
    var = varaux.copy()
    var[criterio1 & criterio2] = np.nan
    
    # Retorna la serie con valores atípicos reemplazados y el número de valores atípicos identificados
    return [var, sum(criterio1 & criterio2)]

In [15]:
# Listado de alimentos y comunidades autónomas para procesar
seleccion_alimentos = df_nacional_filtrado['Alimentos'].unique()
seleccion_ccaa = df_ccaa_filtrado['CCAA'].unique()

# Diccionarios para almacenar las series
dict_series_mensuales_nacional = {}
dict_series_mensuales_ccaa = {}

# Procesar las series a nivel nacional
for alimento in seleccion_alimentos:
    # Filtrar datos por alimento a nivel nacional
    serie_nacional = df_nacional_filtrado[df_nacional_filtrado['Alimentos'] == alimento]
    dict_series_mensuales_nacional[alimento] = serie_nacional

    
# Procesar las series por comunidad autónoma
for ccaa in seleccion_ccaa:
    # Filtrar datos por comunidad autónoma
    serie_ccaa = df_ccaa_filtrado[df_ccaa_filtrado['CCAA'] == ccaa]
    
    # Crear un diccionario para almacenar las series filtradas por alimento para esta CCAA
    dict_series_por_alimento = {}
    
    for alimento in seleccion_alimentos:
        # Filtrar datos por alimento y comunidad autónoma
        serie_alimento_ccaa = serie_ccaa[serie_ccaa['Alimentos'] == alimento]
        if not serie_alimento_ccaa.empty:
            dict_series_por_alimento[alimento] = serie_alimento_ccaa
    
    # Almacenar las series filtradas por alimento en el diccionario principal
    dict_series_mensuales_ccaa[ccaa] = dict_series_por_alimento

In [16]:
#Definimos variables númericas del dataset
var_num=['CONSUMO X CAPITA', 'GASTO X CAPITA', 'PENETRACION (%)','PRECIO MEDIO kg ó litros', 'VALOR (Miles Euros)',
       'VOLUMEN (Miles kg ó litros)']

#Recuento del porcentaje de atipicos de cada variable a nivel nacional
#La proporción de valores atípicos se calcula dividiendo la cantidad de valores atípicos por el número total de filas.
for alimento, serie in dict_series_mensuales_nacional.items():
    resultados_nacional = {x: atipicosAmissing(serie[x])[1] / len(serie) for x in var_num}
    print(alimento)
    print(f'{resultados_nacional}\n')

T.HUEVOS KGS
{'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0033003300330033004, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.07920792079207921}

TOTAL CARNE
{'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.006600660066006601, 'PENETRACION (%)': 0.0165016501650165, 'PRECIO MEDIO kg ó litros': 0.006600660066006601, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

TOTAL PESCA
{'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0033003300330033004, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0033003300330033004, 'VOLUMEN (Miles kg ó litros)': 0.0}

TOTAL LECHE LIQUIDA
{'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

DERIVADOS LACTEOS
{'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0033003300330033004, 'PRECIO MEDIO kg ó litros': 0.0

In [17]:
# Procesar las series en dict_series_mensuales_ccaa
for ccaa, dict_alimentos in dict_series_mensuales_ccaa.items():
    print(f'{ccaa}------------------------------------------------------------------------------------>')
    
    for alimento, serie_alimento in dict_alimentos.items():      
        # Calcular el porcentaje de valores atípicos para cada variable numérica
        resultados_alimento = {}
        for x in var_num:
            if x in serie_alimento.columns:  # Verificar si la columna existe en el DataFrame
                resultados_alimento[x] = atipicosAmissing(serie_alimento[x])[1] / len(serie_alimento)
            else:
                resultados_alimento[x] = None  # Si la columna no existe, almacenar None o algún valor por defecto
        
        # Imprimir los resultados o realizar alguna otra acción
        print(f" Resultados de valores atípicos para {alimento}: {resultados_alimento}\n")

CATALUÑA------------------------------------------------------------------------------------>
 Resultados de valores atípicos para T.HUEVOS KGS: {'CONSUMO X CAPITA': 0.010752688172043012, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.007168458781362007}

 Resultados de valores atípicos para TOTAL CARNE: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.010752688172043012, 'PRECIO MEDIO kg ó litros': 0.0035842293906810036, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL PESCA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.025089605734767026, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0035842293906810036, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL LECHE LIQUIDA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO

 Resultados de valores atípicos para PLATOS PREPARADOS: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

VALENCIA------------------------------------------------------------------------------------>
 Resultados de valores atípicos para T.HUEVOS KGS: {'CONSUMO X CAPITA': 0.006600660066006601, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0033003300330033004}

 Resultados de valores atípicos para TOTAL CARNE: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0033003300330033004, 'PENETRACION (%)': 0.0165016501650165, 'PRECIO MEDIO kg ó litros': 0.0033003300330033004, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL PESCA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.009900990099009901, 'PENETRACION (%)': 0.0, 'PRECIO ME

 Resultados de valores atípicos para T.HUEVOS KGS: {'CONSUMO X CAPITA': 0.0033003300330033004, 'GASTO X CAPITA': 0.0033003300330033004, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.006600660066006601}

 Resultados de valores atípicos para TOTAL CARNE: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.019801980198019802, 'PRECIO MEDIO kg ó litros': 0.006600660066006601, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL PESCA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL LECHE LIQUIDA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valor

 Resultados de valores atípicos para T.HUEVOS KGS: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL CARNE: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.013201320132013201, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL PESCA: {'CONSUMO X CAPITA': 0.0033003300330033004, 'GASTO X CAPITA': 0.0033003300330033004, 'PENETRACION (%)': 0.0033003300330033004, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0033003300330033004, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL LECHE LIQUIDA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de val

 Resultados de valores atípicos para TOTAL CARNE: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL PESCA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL LECHE LIQUIDA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0033003300330033004, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para DERIVADOS LACTEOS: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para PAN: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA'

 Resultados de valores atípicos para T.HUEVOS KGS: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL CARNE: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0033003300330033004, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL PESCA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para TOTAL LECHE LIQUIDA: {'CONSUMO X CAPITA': 0.0, 'GASTO X CAPITA': 0.0, 'PENETRACION (%)': 0.0, 'PRECIO MEDIO kg ó litros': 0.0, 'VALOR (Miles Euros)': 0.0, 'VOLUMEN (Miles kg ó litros)': 0.0}

 Resultados de valores atípicos para DERIVADOS LACTEOS: {'CONSUMO X CAPITA': 0.0, 'GASTO 

Para el análisis del dataset a nivel nacional, observamos que el porcentaje de valores atípicos es muy bajo y afecta únicamente a las series de datos de hortalizas, legumbres, arroz, derivados lácteos, pesca, carne y huevos, es decir, a 7 de las 12 series de estudio. Dado el impacto limitado, podríamos considerar dos enfoques:

1. Mantener los valores atípicos y evaluarlos en el contexto de cada serie para observar cómo afectan los resultados finales en el modelado.

2. Transformar estos valores atípicos en valores faltantes (NaN) y aplicar un tratamiento adecuado para estos datos faltantes.

Para tener las dos opciones, se aplica el tratamiento de los datos atípicos generando un nuevo dataframe de cada dataset.

In [18]:
# Crear un DataFrame vacío para almacenar todas las series tratadas
df_nacional_filtrado_2 = pd.DataFrame()

# Aplicar el tratamiento de valores atípicos a nivel de cada serie específica a nivel nacional
dict_series_mensuales_nacional_tratada = {}

for alimento, serie in dict_series_mensuales_nacional.items():
    serie_tratada = serie.copy()  # Copiar el DataFrame para no modificar el original
    for x in var_num:
        if x in serie.columns:
            # Reemplazar valores atípicos con NaN en la serie específica
            serie_tratada[x] = atipicosAmissing(serie[x])[0]
    dict_series_mensuales_nacional_tratada[alimento] = serie_tratada
    
    # Concatenar cada DataFrame tratado en el DataFrame final
    df_nacional_filtrado_2  = pd.concat([df_nacional_filtrado_2 , serie_tratada], ignore_index=True)

In [19]:
# Crear un DataFrame vacío para almacenar todas las series tratadas
df_ccaa_filtrado_2 = pd.DataFrame()

# Procesar las series en dict_series_mensuales_ccaa
for ccaa, dict_alimentos in dict_series_mensuales_ccaa.items():
    for alimento, serie_alimento in dict_alimentos.items():
        serie_tratada = serie_alimento.copy()  # Copiar el DataFrame para no modificar el original
        
        # Reemplazar valores atípicos con NaN en la serie específica
        for x in var_num:
            if x in serie_alimento.columns:
                serie_tratada[x] = atipicosAmissing(serie_alimento[x])[0]
        
        # Añadir nuevamente una columna para la CCAA
        serie_tratada['CCAA'] = ccaa
        
        # Concatenar cada DataFrame tratado en el DataFrame final
        df_ccaa_filtrado_2 = pd.concat([df_ccaa_filtrado_2, serie_tratada], ignore_index=True)


### DATOS FALTANTES O MISSING

En el análisis previo, utilizando el método .info(), confirmamos que los dos DataFrames resultantes del filtrado no contenían valores nulos. Sin embargo, tras aplicar el análisis de valores atípicos y reemplazar estos valores atípicos con NaN, hemos introducido datos faltantes en los DataFrames.

In [20]:
df_lista=[df_nacional_filtrado_2,df_ccaa_filtrado_2]
# Verificar valores nulos en cada DataFrame
for i, df in enumerate(df_lista, start=1):
    print(f"\nDataFrame {i}:")
    # Contar valores nulos por columna
    nulos_por_columna = df.isnull().sum()
    
    # Filtrar columnas con valores nulos
    columnas_con_nulos = nulos_por_columna[nulos_por_columna > 0]
    
    if not columnas_con_nulos.empty:
        # Mostrar columnas con valores nulos
        print("Columnas con valores nulos:")
        print(columnas_con_nulos)
    else:
        # Indicar que no hay valores nulos
        print("No hay valores nulos en este DataFrame.")


DataFrame 1:
Columnas con valores nulos:
CONSUMO X CAPITA                1
GASTO X CAPITA                  7
PENETRACION (%)                 6
PRECIO MEDIO kg ó litros        4
VALOR (Miles Euros)             1
VOLUMEN (Miles kg ó litros)    47
dtype: int64

DataFrame 2:
Columnas con valores nulos:
CONSUMO X CAPITA               47
GASTO X CAPITA                 52
PENETRACION (%)                74
PRECIO MEDIO kg ó litros       79
VALOR (Miles Euros)            37
VOLUMEN (Miles kg ó litros)    35
dtype: int64


Al revisar los resultados de la proporción de valores perdidos por variable, observamos que ninguna variable tiene una proporción de valores perdidos superior al 50%. Como mencionamos anteriormente, el porcentaje es mínimo. 

In [21]:
# Muestra proporción de valores perdidos por cada VARIABLE (guardo la información)
prop_missingsVars = df_nacional_filtrado_2.isna().sum()/len(df_nacional_filtrado_2)
prop_missingsVars

Alimentos                      0.000000
CONSUMO X CAPITA               0.000275
GASTO X CAPITA                 0.001925
PENETRACION (%)                0.001650
PRECIO MEDIO kg ó litros       0.001100
VALOR (Miles Euros)            0.000275
VOLUMEN (Miles kg ó litros)    0.012926
Año                            0.000000
Mes                            0.000000
Fecha                          0.000000
dtype: float64

In [22]:
# Muestra proporción de valores perdidos por cada VARIABLE (guardo la información)
prop_missingsVars = df_ccaa_filtrado_2.isna().sum()/len(df_ccaa_filtrado_2)
prop_missingsVars

Alimentos                      0.000000
CONSUMO X CAPITA               0.000764
GASTO X CAPITA                 0.000845
PENETRACION (%)                0.001203
PRECIO MEDIO kg ó litros       0.001284
VALOR (Miles Euros)            0.000601
VOLUMEN (Miles kg ó litros)    0.000569
Mes                            0.000000
Año                            0.000000
CCAA                           0.000000
Fecha                          0.000000
dtype: float64

Al analizar series temporales, es crucial tener en cuenta que los valores atípicos pueden revelar patrones importantes o anomalías relevantes. Estos valores atípicos podrían ser indicativos de estacionalidades específicas, cambios bruscos en el comportamiento de la serie, o influencias de factores externos que son significativos para la interpretación de los datos.

En nuestro caso, hemos optado por mantener los datos depurados pero sin tratar los datos atípicos. Esta decisión se ha tomado para preservar la integridad de las series temporales originales y para evitar la introducción de sesgos que podrían resultar de la transformación de estos valores.

En el análisis exploratorio posterior, se generaran representaciones gráficas de las series temporales. Estas visualizaciones permitirán observar desviaciones importantes y patrones que podrían no ser evidentes en el análisis de datos numéricos. Los gráficos pueden ser una herramienta valiosa para identificar y contextualizar los valores atípicos, facilitando una mejor comprensión del comportamiento general de la serie.

Para futuras investigaciones, se recomienda llevar a cabo un análisis más detallado de cada anomalía identificada. Este análisis puede ayudar a entender el origen de los valores atípicos y a determinar si son el resultado de eventos únicos, errores de medición, o cambios estructurales en la serie temporal.

### GUARDAR DEPURADOS

Los datos se han guardado en dos formatos: .csv y .pickle. El formato .pickle se utilizará en los próximos notebooks debido a su eficiencia para manejar objetos en Python, mientras que el .csv se ofrece como una opción para trabajar en entornos como Excel o aplicaciones de visualización de datos como Tableau, Power BI y Qlik. Esto asegura que los datos sean accesibles en diversas plataformas y aplicaciones.

In [23]:
# Guarda los DataFrames en archivos pickle en el directorio de salida y csv. El csv lo aplicaremos en Tableau.
datosNacional_pk = os.path.join(directorioSalida, 'df_nacional_filtrado.pkl')
datosCcaa_pk= os.path.join(directorioSalida, 'df_ccaa_filtrado.pkl')

df_nacional_filtrado.to_pickle(datosNacional_pk)
df_ccaa_filtrado.to_pickle(datosCcaa_pk)

datosNacional_csv = os.path.join(directorioSalida, 'df_nacional_filtrado.csv')
datosCcaa_csv = os.path.join(directorioSalida, 'df_ccaa_filtrado.csv')

# Guarda los DataFrames en archivos CSV
df_nacional_filtrado.to_csv(datosNacional_csv, index=False)
df_ccaa_filtrado.to_csv(datosCcaa_csv, index=False)